# Preparation

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import IPython
from scipy import signal

from Echo_cancellation_utils import EchoCancellation

%matplotlib widget

In [2]:
# Load the signals
EC = EchoCancellation()
FS = EC.FS
x = EC.x
y = EC.y

# Exercise

We have received the following audio file:

In [3]:
# Plot
plt.close('all')
fig, ax = plt.subplots(1, 1, figsize=(8, 2))
ax.plot(np.linspace(0, len(y)/FS, len(y)), y, linewidth=0.7)
ax.set_title('Audio file'); ax.set_xlabel('Time [s]'); plt.tight_layout(); plt.show()
# Create audio widget
display(IPython.display.Audio(y, rate=FS))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As can be heard, the audio has some undesirable echoes. The aim of this exercise is to remove the echo and to obtain a clean signal. The audio file is stored numerically in the signal $y[n]$ whose size is $N=185140$ and has a sampling frequency $F_s=48000\;Hz$.

* Q: Determince the length $T$ of the signal **in seconds** (up to two digits)?

In [4]:
EC.Q1

We consider the following model for echo:

$$y[n]=x[n]+ax[n-n_0]\;\;\;\;\;\;\;\;\;\;(1)$$
where
* $x[n]$ is the clean signal
* $a \in {\rm I\!R}$ with $|a|\lt 1$ is the attenuation factor
* $n_0 \in \mathbb{N}$ is the shift

Let us recall that for a discrete signal $x[n]$, its auto correlation function is defined as

$$\mathrm{R_x}[m]=\sum_{n \in \mathbb{Z}} x[n]x[m+n]$$

Using $(1)$, we can compute the auto correlation function of $y[n]$, denoted by $\mathrm{R_y}[m]$, as 

$$\mathrm{R_y}[m] = a_0\mathrm{R_x}[m] + a_1\mathrm{R_x}[m-m_1] + a_2\mathrm{R_x}[m-m_2],$$

where $a_0$, $a_1$, $a_2$ $\in {\rm I\!R}$ and $n_1$, $n_2$ $\in \mathbb{Z}$ depend on the model parameters $(a, n_0)$.

* Q: For the special case $n_0=5$ and $a=\frac{1}{2}$, give the values of $a_0$, $a_1$, $a_2$ $\in {\rm I\!R}$ as well as $m_1$, $m_2$ $\in \mathbb{Z}$ assuming the convention $m_1 \lt m_2$.

In [5]:
EC.Q2

To determine the unknown parameters $(a, n_0)$, we rely on the assumption that $\mathrm{R_x}[m]$ has only one major peak at $m=0$ and it rapidly decreases as $m$ grows (see the following figure for an illustration).

In [12]:
# Calculate auto correlation of x
corr_x = signal.correlate(x, x, mode='same')
corr_x = corr_x / np.max(np.abs(corr_x)) # Normalize

# Plot
plt.close('all')
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(np.linspace(-len(corr_x)//2, len(corr_x)//2-1, len(corr_x)), corr_x, linewidth=0.5)
ax.grid(); ax.set_title('$R_x[m]$'); ax.set_xlabel('$m$'); plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Using this assumption, we can claim that $\mathrm{R_y}[m]$ has a main peak at $m=0$ with height $a_0\mathrm{R_x}[0]$ and two side peaks at $m=m_1$ and $m=m_2$ with heights $a_1\mathrm{R_x}[0]$ and $a_2\mathrm{R_x}[0]$, respectively.

Now let us look at the auto correlation function $\mathrm{R_y}[m]$.

In [11]:
# Calculate auto correlation of y
corr_y = signal.correlate(y, y, mode='same')
corr_y = corr_y / np.max(np.abs(corr_y)) # Normalize

# Peak detection
peaks, _ = signal.find_peaks(corr_y, height=0.3*np.max(corr_y), distance=1000)
peaks_adjusted = peaks - len(corr_y)//2

# Plot
plt.close('all')
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
ax.plot(np.linspace(-len(corr_y)//2, len(corr_y)//2-1, len(corr_y)), corr_y, linewidth=0.5)
for i in range(len(peaks)):
    ax.plot(peaks_adjusted[i], corr_y[peaks[i]], 'rx')
    ax.text(peaks_adjusted[i], corr_y[peaks[i]]+0.05, f'({peaks_adjusted[i]}, {corr_y[peaks[i]]:.3f})', horizontalalignment='center', color='red')
ax.set_ylim([ax.get_ylim()[0], 1.15]); ax.grid(); ax.set_title('$R_y[m]$'); ax.set_xlabel('$m$'); plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

* Q: Using this plot and the above description, estimate the unknown parameters $a \in {\rm I\!R}$ and $n_0 \in \mathbb{N}$.

In [13]:
EC.Q3

* Q: Using $n_0$ and the sample frequency $F_s$, determine the echo time $T_e$ **in seconds**.

In [14]:
EC.Q4

Based on your answer, we now apply the following inverse filter to $y[n]$:

$$y[n] \longrightarrow \boxed{\frac{1}{1 + az^{-n_0}}} \longrightarrow \tilde{x}[n]$$

In [15]:
# Get a and n0 from the answer above
a = EC.Q3.children[0].children[0].value
n0 = int(EC.Q3.children[1].children[0].value)

# Filter signal
den = np.zeros(n0 + 1)
den[0] = 1.
den[-1] = a
x_est = signal.lfilter(np.array([1.]), den, y)

Let's hear $\tilde{x}[n]$:

In [16]:
# Plot signal
plt.close('all')
fig, ax = plt.subplots(1, 1, figsize=(8, 2))
ax.plot(np.linspace(0, len(x_est)/FS, len(x_est)), x_est, linewidth=0.7)
ax.set_title(r'$\tilde{x}[n]$'); ax.set_xlabel('Time [s]'); plt.tight_layout(); plt.show()
# Create audio widget
display(IPython.display.Audio(x_est, rate=FS))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<div class='alert alert-success'>
    <b>If you hear a lie, then Congratulations!</b>
<div>